# MATH 405/607 

# Chaos In The Context Of Dynamical Systems

Callum Hepworth, Sandy Luo, Jed Yeo


## Content

1) Introduction to Chaos Theory
2) Strange Attractor & Lorenz Problem
3) Popular Examples of Chaos
4) Studying Chaos Numerically (ODEs)

## Introduction

- Chaos theory 
    - "Study of random or unpredictable behaviour in systems governed by deterministic laws" [1]
    - Example: Pinball machine
- Applications
    - Turbulent flow
    - Chemical reactions
    - Planetary motion

- Dynamical systems
    - Highly sensitive to initial conditions
    - i.e, adjusting initial conditions by some amount leads to interesting changes in the solution
    - Behavior of dynamical chaotic systems appears random
    - This is not the case - these systems are deterministic (future dynamics are fully defined by initial conditions)

## Lorenz Heat Problem

- Chaos first observed by Edward Lorenz in 1963 
- The problem: atmospheric convection (three ODEs):

$$\frac{dx}{dt} = \sigma(y-x), \frac{dy}{dt}=x(\rho-z)-y, \frac{dz}{dt}=xy-\beta z $$


- These three equations relate the properties of a 2D fluid layer uniformly warmed from below and cooled from above
- $x$ is proportional to rate of convection, $y$ to horizontal temperature gradient, $z$ to the vertical gradient
- Constants depend on physical properties of the system



In [ ]:
# We take σ = 10, ρ = 28, β = 8/3
include("math405.jl")
using OrdinaryDiffEq, Plots
function lorenz(du, u, p, t)
    du[1] = 10.0*(u[2]-u[1])
    du[2] = u[1]*(28.0-u[3]) - u[2]
    du[3] = u[1]*u[2] - (8/3)*u[3]
end

In [ ]:
u0 = [-15.0;-15.0;20.0]
tspan = (0.0,100.0)
prob = ODEProblem(lorenz,u0,tspan)
sol0 = solve(prob, Tsit5())
plot(sol0, vars=(1,2,3))

In [ ]:
u1 = [-15.0;-15.0;20.0001]
prob = ODEProblem(lorenz,u1,tspan)
sol1 = solve(prob, Tsit5())
plot(sol1, vars=(1,2,3))

- These butterfly-like shapes reveal the structure of the "stranger attractor" for the Lorenz problem
- A mere adjustment of $1\times10^{-4}$ in the third initial condition yields a vastly different solution space

- There isn't a universally accepted definition of chaos, but in general a chaotic problem must loosely follow these guidelines:
   - 1) be sensitive to initial conditions
   - 2) be topologically transitive
   - 3) have dense periodic orbits
- It can be shown that the last two imply the first; we will not discuss these conditons as we are mainly interested in pertubations of the initial condition(s).

In [ ]:
@gif for m = 1:50
    u0 = [-15.0;-15.0;20]
    tspan = (0.0,m*2.0)
    prob = ODEProblem(lorenz,u0,tspan)
    sol0 = solve(prob, Tsit5())
    plot(sol0,vars=(1,3))
end

In [ ]:
@gif for m = 1:50
    u1 = [-15.0;-15.0;20.0001]
    tspan = (0.0,m*2.0)
    prob = ODEProblem(lorenz,u1,tspan)
    sol1 = solve(prob, Tsit5())
    plot(sol1,vars=(1,3),color="orange")
end

In [ ]:
plot(sol0, vars=(1,3), xlabel=L"x", ylabel=L"z", label="\$ z(0)=20 \$", title="\$ xz\$ plane with differing ICs")
plot!(sol1, vars=(1,3), label="\$ z(0)=20.0001 \$")

## Strange Attractors

- Lorenz's choices for $\beta = \frac{8}{3}$, $\sigma = 10$, and $\rho = 28$ were intentional.

- With these parameters almost all IC's tend toward an invariant set of states as the system evolves numerically.

- For a general ODE this set of states is called an $\textbf{attractor}$, and if these states form a fractal they are called a $\textbf{strange attractor}$.

    - Because it's a notable example, the strange attractor that emerges from the Lorenz system is dubbed the $\textbf{Lorenz attractor}$.

- While the Lorenz attractor is a notable example, these behaviours emerge from a variety of dynamical systems.

- Most often, but not always, strange attractors emerge from chaotic systems.

- As Jed mentioned in the introduction, a necessary condition of a chaotic system is that the behaviour of the system is heavily dependent on the choice of initial conditions.

## Sensitivity to Initial Conditions

- We can visualize this initial condition dependence by taking a dynamical system that is known to be chaotic, i.e. the Lorenz system with $\beta = \frac{8}{3}$, $\sigma = 10$, and $\rho = 28$, and perturb $z_0$.

In [ ]:
u1 = [-15.0;-15.0;20.0]
prob = ODEProblem(lorenz,u1,tspan)
sol_unperturbed = solve(prob, Tsit5())

u1 = [-15.0;-15.0;20.00001]
prob = ODEProblem(lorenz,u1,tspan)
sol_perturbed = solve(prob, Tsit5())

plot(sol_unperturbed,vars=(1), color=1, xlims=(0, 20))
plot!(sol_perturbed,vars=(1), color=2, xlims=(0, 20))

In [ ]:
plot(sol_unperturbed,vars=(2), color=1, xlims=(0, 20))
plot!(sol_perturbed,vars=(2), color=2, xlims=(0, 20))

In [ ]:
plot(sol_unperturbed,vars=(3), color=1, xlims=(0, 20))
plot!(sol_perturbed,vars=(3), color=2, xlims=(0, 20))

- Despite having $z_0$ differ by one part in $10^6$, the systems diverge wildly after the $\approx 13^{th}$ timestep.
- To briefly quantify this behaviour, we note that an initial state difference of $10^{-5}$ in $z_0$ has grown by six orders of magnitude over $15$ time steps.

## Can we quantify this rate of divergence?

- To quantify trends in the sensitivity of chaotic dynamical systems to initial conditions, it is useful to first visualize the rate of divergence over time.

- We can do so by plotting the $l$-2 norm of the difference between the perturbed and unperturbed states as a function of time. 
    
    - This has been done for the Lorenz system below.


In [ ]:
TT = 1:1:500
err = @. sqrt((sol_perturbed[1, TT]-sol_unperturbed[1, TT])^2+(sol_perturbed[2, TT]-sol_unperturbed[2, TT])^2+(sol_perturbed[3, TT]-sol_unperturbed[3, TT])^2)

TT = (1:1:500) * (40 / 700)
plot(TT,err, yaxis=:log, xlabel="t", ylabel=L"|| \hat{y}(t) - y(t) ||_{2}", title=L"l" * "-2 Norm of Perturbed and Unperturbed State Difference vs. Time", label="", size=(800,400))

- Applying this visualization reveals two distinct regions in the growth of the perturbations in our initially perturbed and unperturbed systems.

- When T > 15, the $l$-2 error appears to reach an equilibrium and stops growing.

- When T < 15, the $l$-2 error appears to grow exponentially. 

- In general, the rate of growth in this initial exponential regime for a particular ODE is called it's $\textbf{Lyapunov exponent}$. 

- This exponent is defined as $\approx 0.9052$ for the Lorenz system. 

- Plotting an exponential function with this Lyapunov exponent looks as follows.

In [ ]:
TT = 1:1:500
err = @. sqrt((sol_perturbed[1, TT]-sol_unperturbed[1, TT])^2+(sol_perturbed[2, TT]-sol_unperturbed[2, TT])^2+(sol_perturbed[3, TT]-sol_unperturbed[3, TT])^2)

TT = (1:1:500) * (40 / 700)
plot(TT,err, yaxis=:log, xlabel="t", ylabel=L"|| \hat{y}(t) - y(t) ||_{2}", title=L"l" * "-2 Norm of Perturbed and Unperturbed State Difference vs. Time",legend=:bottomright, label="perturbation norm", size=(800,400))

TT = (1:1:300) * (40 / 700)
plot!(TT, 0.0001 *exp.(TT * 0.9052), label=L"\lambda = 0.9052")

- We can say that perturbations in Lorenz systems typically grow at a rate of:

\begin{align*}
    \| \hat{y}(t) - y(t) \|_2 &\approx Ce^{0.9052t}
\end{align*}

In general, the formal definition of a Lyapunov exponent is given by:

\begin{align*}
    \lambda &= \underset{t \rightarrow \infty}{lim} \underset{\| \delta y(0) \| \rightarrow 0}{limsup} t^{-1} log \frac{\| \delta y(t) \|}{\| \delta y(0) \|},
\end{align*}

where:

\begin{align*}
    \delta y(t) &= \hat{y}(t) - y(t)
\end{align*}

## Popular Examples of Chaos

### Rossler Equations

The Rossler Equations:

 $$u'=-v-w$$, $$v'=u+av$$, $$w'=b+w(u-c)$$

In [ ]:
@gif for m = 2:0.1:10
    function Rossler(du,u,p,t)
        du[1] = -u[2]-u[3]
        du[2] = u[1]+0.2*u[2]
        du[3] = 0.2+u[3].*(u[1]-m)
    end

    u0 = [2;0;0]
    tspan = (0,300.0)
    prob = ODEProblem(Rossler,u0,tspan)
    sol = solve(prob, Tsit5())
    
    p1=scatter(sol[1,200:400],sol[2,200:400],sol[3,200:400])
    p2=scatter(sol[1,200:400],sol[2,200:400])
    
    plot(p1, p2, legend=false,title=m)
end

### Nonlinear Forced Oscillator
This is a nonautonomous equation, or a chaotic nonlinear forced oscillator. 

The equation is

$$y''+\frac{1}{4}y'-y+y^3=0.4cos(t)$$

The forcing function makes this system equivalent to a first-order autonomous system. Thus the dimension is great enough for chaos to be a possibility, and a computation confirms its appearance.

In [ ]:
function nonlinear(du,u,p,t)
    du[1] = u[2]
    du[2] = -0.25*u[2]+u[1]-u[1].^3+0.4*cos(t)
end

u0 = [0;0]
tspan = (0.0,300.0)
prob = ODEProblem(nonlinear,u0,tspan)
sol = solve(prob, Tsit5())
plot(sol)

In [ ]:
plot(sol[1,:],sol[2,:])

### Chaos in a Food Web

This is a simple of the interactions between rabbits and foxes in their population. 

We consider the population of rabbit as $u(t)$, the popluation of foxes $v(t)$ and population of carrots $c(t)$.

We can then produce a reasonable model for rabbits consuming carrots, foxes consuming rabbits, and neither animal species can be sustained without food.

$$
c'=c(1-c)-f_1(c)u, u'=f_1(c)u-f_2(u)v-d_1u, v'f_2(u)v-d_2v
$$

## Studying Chaos Numerically

### Analyzing Rossler Numerically

### Rossler Equations

The Rossler Equations:

 $$u'=-v-w,$$ $$v'=u+av,$$ $$w'=b+w(u-c)$$
 
We set the initial condition as $u(0)=2$, $v(0)=w(0)=0$
 
Also, we set $a=b=\frac{1}{5}$ arbitrarily and $c$ is a parameter.

We want to study how the values of $c$ would change the behaviour of the solutions.

In [ ]:
function Rossler(du,u,p,t)
    du[1] = -u[2]-u[3]
    du[2] = u[1]+0.2*u[2]
    du[3] = 0.2+u[3].*(u[1]-2)
end

u0 = [2;0;0]
tspan = (0,300.0)
prob = ODEProblem(Rossler,u0,tspan)
sol = solve(prob, Tsit5())
p1=plot(sol,vars=(1,2,3))
p2=plot(sol,vars=(1,2))
plot(p1, p2, legend=false,title="c=2,after t>200")

In [ ]:
@gif for m = 1:60
    u0 = [2;0;0]
    tspan = (0.0,m*0.6)
    prob = ODEProblem(Rossler,u0,tspan)
    sol = solve(prob, Tsit5())
    p1=plot(sol,vars=(1,2,3))
    p2=plot(sol,vars=(1,2))
    plot(p1, p2, legend=false,title="c=2")
end


In [ ]:
u0 = [2;0;0]
tspan = (0,300.0)
prob = ODEProblem(Rossler,u0,tspan)
sol = solve(prob, Tsit5())
p1=plot(sol,vars=(1,2,3))
p2=plot(sol,vars=(1,2))
plot(p1, p2, legend=false,title="c=2,after t>200")
plot(sol)

In [ ]:
p1=scatter(sol[1,526:791],sol[2,526:791],sol[3,526:791])
p2=scatter(sol[1,526:791],sol[2,526:791])
plot(p1, p2, legend=false,title="c=2,after t>200")

As the parameter c increases, a bifurcation takes place. At around $c = 2.8$, the limit cycle undergoes period doubling, with the trajectory unfolding into a double loop.

In [ ]:
function Rossler1(du,u,p,t)
    du[1] = -u[2]-u[3]
    du[2] = u[1]+0.2*u[2]
    du[3] = 0.2+u[3].*(u[1]-3.5)
end
u0 = [2;0;0]
tspan = (0,300.0)
prob = ODEProblem(Rossler1,u0,tspan)
sol = solve(prob, Tsit5())
p1=scatter(sol[1,526:791],sol[2,526:791],sol[3,526:791])
p2=scatter(sol[1,526:791],sol[2,526:791])
plot(p1, p2, legend=false,title="c=3.5,after t>200")

The following figure shows how the limit cycle undergoes period doubling.

In [ ]:
@gif for m = 1:0.1:4.0
    function Rossler(du,u,p,t)
        du[1] = -u[2]-u[3]
        du[2] = u[1]+0.2*u[2]
        du[3] = 0.2+u[3].*(u[1]-m)
    end

    u0 = [2;0;0]
    tspan = (0.0,300.0)
    prob = ODEProblem(Rossler,u0,tspan)
    sol = solve(prob, Tsit5())
    
    p1=scatter(sol[1,200:400],sol[2,200:400],sol[3,200:400])
    p2=scatter(sol[1,200:400],sol[2,200:400])
    plot(p1, p2, legend=false,title=m)
end

The phenomenon is called period doubling by Mitchell Feigenbaum.

It illustrates a route to chaos.

In this question, as the value of $c$ increases further, the period doubles again and again, infinitely often.

Finally, for $c$ greater than about $4.2$, the system is chaotic. 

The following images show the chaotic regime, with the orbits settling down not to a limit cycle but to a strange attractor.

### Chaos in a Food Web

$$
c'=c(1-c)-f_1(c)u, u'=f_1(c)u-f_2(u)v-d_1u, v'f_2(u)v-d_2v
$$

where $$f_1(z)=\frac{a_1z}{(1+b_1z)}, f_2(z)=\frac{a_2z}{(1+b_2z)}$$

We fix the value of $a_1=5, a_2=0.1, b_2=2, d_1=0.4, d_2=0.01$ and vary the parameter $b_1$.

$b_1$ describe the effect of competition between rabbits.

As we fix $b_1=2.5$, this is a nonchaotic system.

In [ ]:
function fun(z,a,b)
    res=a*z/(1+b*z)
    return res
end

a1=5
a2=0.1
b1=2.5
b2=2

function rab(du,u,p,t)
    du[1] = u[1].*(1-u[1])-fun(u[1],a1,b1)*u[2]
    du[2] = fun(u[1],a1,b1)*u[2]-fun(u[2],a2,b2)*u[3]-0.4*u[2]
    du[3] = fun(u[2],a2,b2)*u[3]-0.01*u[3]
end

u0 = [0.4;1;9]
tspan = (0.0,3000.0)
prob = ODEProblem(rab,u0,tspan)
sol = solve(prob, Tsit5())
p1=scatter(sol[1,400:end],sol[2,400:end],sol[3,400:end])
p2=scatter(sol[1,400:end],sol[2,400:end])
    
plot(p1, p2, legend=false,title="b1=2.5")

In [ ]:
plot(sol)

In [ ]:
When we change the value of $b_1$ to $6$, the system becomes chaotic

In [ ]:
b1=6

function rab(du,u,p,t)
    du[1] = u[1].*(1-u[1])-fun(u[1],a1,b1)*u[2]
    du[2] = fun(u[1],a1,b1)*u[2]-fun(u[2],a2,b2)*u[3]-0.4*u[2]
    du[3] = fun(u[2],a2,b2)*u[3]-0.01*u[3]
end

u0 = [0.4;1;9]
tspan = (0.0,3000.0)
prob = ODEProblem(rab,u0,tspan)
sol = solve(prob, Tsit5())
plot(sol,vars=(1,2,3))

In [ ]:
plot(sol)

- The process of solving chaotic problems numerically is no different from that of a regular ODE system
- e.g. the Lorenz problem is simply a system of 3 IVPs

$$\frac{dx}{dt} = \sigma(y-x), \frac{dy}{dt}=x(\rho-z)-y, \frac{dz}{dt}=xy-\beta z $$

- The process is similar to solving any of the other ODEs we've solved in class
- Throughout the presentation we opted to use the $\texttt{Tsit5}$ method to solve our systems
- Let's try our own implementation of Kutta third-order method and compare it to Improved Euler

- Butcher tableau:
$$\begin{array}{c|ccc} 
    0 &   &   &   \\ 
    1/2 & 1/2 &   &   \\ 
    1 & -1 & 2 &     \\ 
    \hline 
      & 1/6 & 2/3  & 1/6 
\end{array}$$ 

-  Improved Euler method
$$
    U_{n+1} = U_n + h \frac{f(t_n, U_n) + f\big(t_{n+1}, U_n + h f(t_n, U_n)\big)}{2}
$$ 
- $O(h^2)$ accuracy, 2 function evaluations per step

In [ ]:
function kutta3(f, u0, h, Tf)
    t = 0.0:h:Tf 
    # make sure that h and Tf are compatible!
    @assert t[end] ≈ Tf     
    U = zeros(length(u0), length(t))
    U[:, 1] = u0 
    for tn = 1:length(t)-1
        k1 = h * f(t[tn], U[:, tn])
        k2 = h * f(t[tn] + 0.5 * h, U[:, tn] + 0.5 * k1)
        k3 = h * f(t[tn] + 1 * h, U[:, tn] - 1 * k1 + 2 * k2)
        U[:, tn+1] = U[:, tn] + (1/6)*k1 + (2/3)*k2 + (1/6)*k3
    end
    return U, t 
end

function ieuler(f, u0, h, T)
    t = 0.0:h:T 
    U = zeros(length(u0), length(t)); U[:, 1] = u0 
    for n = 2:length(t) 
        F = f(t[n-1], U[:,n-1])
        U[:,n] = U[:,n-1] + 0.5 * h * (F + f(t[n], U[:,n-1] + h * F))
    end 
    return U, t
end

In [ ]:
f(t, u) = [10.0*(u[2]-u[1]), u[1]*(28.0-u[3]) - u[2], u[1]*u[2] - (8/3)*u[3]]
f0 = [-15.0, -15.0, 20.0]
rksoln, t_rk = kutta3(f, f0, 0.005, 30.0)
ieulersoln, t_e = ieuler(f, f0, 0.005, 30.0)
    
plot(t_rk, rksoln[1,:], xlabel="\$ t\$", ylabel="\$ x(t)", title="RK3 vs. Euler for Lorenz Problem \$x(t), h=0.0005\$", label="RK3")
plot!(t_e, ieulersoln[1,:], label="Improved Euler")

- The solutions appear to diverge from each other around 7 time steps
- Let's vary our step size and evaluate the value of the solution after a lengthy period of time

In [ ]:
times = 500:50:2000
final_rk = []
final_ie = []

for time in times
    rksoln, t_rk = kutta3(f, f0, 1/time, 1000.0)
    ieulersoln, t_e = ieuler(f, f0, 1/time, 1000.0)
    append!(final_rk, rksoln[1, end])
    append!(final_ie, ieulersoln[1, end])
end

hs = []
for time in times
    append!(hs, 1/time)
end

scatter(hs, final_rk, title="Value of \$x(t)\$ at \$t=1000.0\$ vs. \$h\$", xlabel=L"h", ylabel="\$x(t)\$", label="\$x(t)\$ w/ RK3")
scatter!(hs, final_ie, label="\$x(t)\$ w/ IE")

- Using both the third order Runge-Kutta method and improved Euler, there appears to be no correlation between the step size and the "final value" (i.e. solution value after some time period)
- Going to smaller step sizes doesn't guarantee uniform convergence
- Due to the butterfly-effect and floating point operations, numerical approaches cannot give a reliable long-term simulation of chaotic dynamic systems [2]
- This is simply an implication of using numerics to evaluate chaotic problems

## Influence of Round-Off Errors

- When dealing with systems that are hypersensitive to initial perturbations, it is natural to want to investigate the impacts that round-off errors, an unfortunate byproduct of numerical computing, can have on the reliability of their simulation. 

- We will now examine these complications in the case of our pedagogical example of the Lorenz system, generalizing the example that we provided previously.

- In our previous example, we varied the step size in our numerical solver of the Lorenz system with parameters designed to elicit chaotic behaviour and observed a scattered distribution of end points. 

- In order to clearly observe the effects of round-off errors on the evolution of our system, it would be ideal if we had a limited number of well defined end points, between which we might observe oscillation as a function of step size.

- Conveniently the Lorenz system has such a property when $\rho$ is chosen as $1 < \rho < 24.74$, with $\beta = \frac{8}{3}$ and $\sigma = 10$. It is well known that with these initial conditions, fixed point attractors emerge at:

\begin{gather*}
x = \sqrt{\beta(\rho - 1)}, y = \sqrt{\beta(\rho-1)}, z = \rho - 1, \\
and \\
x = -\sqrt{\beta(\rho - 1)}, y = -\sqrt{\beta(\rho-1)}, z = \rho - 1
\end{gather*}

- To leverage this regime we will select $\rho = 23$, with $\beta$ and $\sigma$ from before.

- It has been shown that the 4th-order Runge-Kutta method with double precision exhibits a particular sensitivity to the choice of step-size $h$ [2].

- Despite efforts to reduce $h$ and thus shrink truncation errors, final values over identical time horizons consistently fluctate between our two expected fixed points. 

In [ ]:
display("image/png", read("chaotic_fluctuation.png"))

- Even when directly isolating round-off errors as the primary source of uncertainty by taking the truncation error to the $M$th-order, with $M > 30$, fluctuations in final values emerge in compute platforms that leverage multiple concurrent processes. 

- In order to avoid the uncertainty introduced by numerically solving the Lorenz equation, and indeed chaotic systems in general, it is crucial that both truncation and round-off errors be decreased simultaneously. 

- In fact, using the Mth-order Taylor series method with $M \geq 130$ with 512-digit precision, numerical agreement can be achieved over the entire time interval even when accounting for varying numbers of computational processes and results obtained across different machines.

- This relationship is illustrated clearly in the following plot. In this case the initial condition $(5,5,10)$ was was selected with $\rho = 23$ and $\beta$ and $\sigma$ from the Lorenz system. 

In [ ]:
display("image/png", read("stability.png"))

- It is crucial to once again consider that this result was obtained on the Lorenz system with $\textit{non-chaotic}$ initial conditions.

- To eliminate troublesome round-off and truncation errors in this regime requires extreme levels of precision.

- In one instance researchers required the use of a $3500$th-order Taylor expansion method with $4180$-point precision to achieve a convergent numerical solution to the chaotic Lorenz equation over a large time horizon $([0, 10000])$.

- It is clear that numerically accurate simulations of chaotic dynamical solutions are particularly difficult to develop. We are eager to explore recent advances in addressing this issue more deeply.

# References

- [1] - https://en.wikipedia.org/wiki/Chaos_theory
- [2] - https://www.researchgate.net/publication/318488232_Influence_of_round-off_errors_on_the_reliability_of_numerical_simulations_of_chaotic_dynamic_systems